In [2]:
from keras.preprocessing.image import load_img,img_to_array

target_image_path = "C:\\Users\\shivu\\bopro\\deeplearning\\style transfer\\results\\content.jpeg"
style_reference_img_path = "C:\\Users\\shivu\\bopro\\deeplearning\\style transfer\\results\\style2.jpg"

width,height = load_img(target_image_path).size
img_height = 400
img_width = int(width*img_height / height)

Using TensorFlow backend.


In [3]:
img_width

400

In [4]:
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
  img = load_img(image_path,target_size=(img_height,img_width))
  img = img_to_array(img)
  img = np.expand_dims(img,axis=0)
  img = vgg19.preprocess_input(img)
  return img
  
  

In [5]:
preprocess_image(target_image_path).shape

(1, 400, 400, 3)

In [6]:
def deprocess_image(x):
  x[:,:,0]+=103.939
  x[:,:,1]+=116.779
  x[:,:,2]+=123.68
  
  x = x[:,:,::-1]
  
  x = np.clip(x,0,255).astype('uint8')
  
  return x

In [7]:
from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_img_path))
combination_image = K.placeholder((1,img_height,img_width,3))

input_tensor = K.concatenate([target_image,style_reference_image,combination_image],axis=0)

model = vgg19.VGG19(input_tensor=input_tensor,weights="imagenet",include_top=False)

print("Model loaded")


Model loaded


In [8]:
def content_loss(base,combination):
  return K.sum(K.square(base-combination))

In [9]:
def gram_matrix(x):
  features = K.batch_flatten(K.permute_dimensions(x,(2,0,1)))
  gram = K.dot(features,K.transpose(features))
  return gram

In [10]:
def style_loss(style,combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  
  channels = 3
  size = img_height * img_width
  
  return K.sum(K.square(S-C)) / (4. * (channels ** 2) * (size ** 2))


In [11]:
def total_variation_loss(x):
  a = K.square(
x[:, :img_height - 1, :img_width - 1, :] -
x[:, 1:, :img_width - 1, :])
  b = K.square(
x[:, :img_height - 1, :img_width - 1, :] -
x[:, :img_height - 1, 1:, :])
  return K.sum(K.pow(a + b, 1.25))

In [12]:
output_dict = dict([(layer.name,layer.output) for layer in model.layers])


In [13]:
output_dict

{'input_1': <tf.Tensor 'concat:0' shape=(3, 400, 400, 3) dtype=float32>,
 'block1_conv1': <tf.Tensor 'block1_conv1/Relu:0' shape=(3, 400, 400, 64) dtype=float32>,
 'block1_conv2': <tf.Tensor 'block1_conv2/Relu:0' shape=(3, 400, 400, 64) dtype=float32>,
 'block1_pool': <tf.Tensor 'block1_pool/MaxPool:0' shape=(3, 200, 200, 64) dtype=float32>,
 'block2_conv1': <tf.Tensor 'block2_conv1/Relu:0' shape=(3, 200, 200, 128) dtype=float32>,
 'block2_conv2': <tf.Tensor 'block2_conv2/Relu:0' shape=(3, 200, 200, 128) dtype=float32>,
 'block2_pool': <tf.Tensor 'block2_pool/MaxPool:0' shape=(3, 100, 100, 128) dtype=float32>,
 'block3_conv1': <tf.Tensor 'block3_conv1/Relu:0' shape=(3, 100, 100, 256) dtype=float32>,
 'block3_conv2': <tf.Tensor 'block3_conv2/Relu:0' shape=(3, 100, 100, 256) dtype=float32>,
 'block3_conv3': <tf.Tensor 'block3_conv3/Relu:0' shape=(3, 100, 100, 256) dtype=float32>,
 'block3_conv4': <tf.Tensor 'block3_conv4/Relu:0' shape=(3, 100, 100, 256) dtype=float32>,
 'block3_pool': <t

In [17]:
content_layer = "block5_conv2"
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

total_variation_weights = 1e-4
style_weight = 1.
content_weight = 0.025


In [18]:
loss = K.variable(0.)
layer_features = output_dict[content_layer]


In [19]:
layer_features

<tf.Tensor 'block5_conv2/Relu:0' shape=(3, 25, 25, 512) dtype=float32>

In [20]:
target_image_features = layer_features[0,:,:,:]

In [21]:
target_image_features

<tf.Tensor 'strided_slice:0' shape=(25, 25, 512) dtype=float32>

In [22]:
combination_features  = layer_features[2,:,:,:]

loss += content_weight*content_loss(target_image_features,combination_features)

In [23]:
loss

<tf.Tensor 'add:0' shape=() dtype=float32>

In [24]:
for layer_name in style_layers:
  layer_features = output_dict[layer_name]
  style_reference_features = layer_features[1,:,:,:]
  combination_features = layer_features[2,:,:,:]
  
  sl = style_loss(style_reference_features,combination_features)
  
  loss+=(style_weight/len(style_layers)) * sl
  
loss+=total_variation_weights * total_variation_loss(combination_image)

In [25]:
loss

<tf.Tensor 'add_7:0' shape=() dtype=float32>

In [26]:
grads = K.gradients(loss, combination_image)[0]

fetch_loss_and_grads = K.function([combination_image], [loss, grads])

class Evaluator(object):
  
  def __init__(self):
    self.loss_value = None
    self.grads_values = None
  def loss(self, x):
    assert self.loss_value is None
    x = x.reshape((1, img_height, img_width, 3))
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    self.loss_value = loss_value
    self.grad_values = grad_values
    return self.loss_value
  def grads(self, x):
    assert self.loss_value is not None
    grad_values = np.copy(self.grad_values)
    self.loss_value = None
    self.grad_values = None
    return grad_values
evaluator = Evaluator()

In [27]:
from scipy.optimize import fmin_l_bfgs_b
import matplotlib.pyplot as plt
import time
result_prefix = 'my_result'
iterations = 20
x = preprocess_image(target_image_path)
x = x.flatten()

for i in range(iterations):
  print('Start of iteration', i)
  start_time = time.time()
  x, min_val, info = fmin_l_bfgs_b(evaluator.loss,x,fprime=evaluator.grads,maxfun=20)
  print('Current loss value:', min_val)
  img = x.copy().reshape((img_height, img_width, 3))
  img = deprocess_image(img)
  fname = result_prefix + '_at_iteration_%d.png' % i
  plt.imsave(fname, img)
  print('Image saved as', fname)
  end_time = time.time()
  print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0


KeyboardInterrupt: 

In [24]:
import cv2
import matplotlib.pyplot as plt


In [25]:
from imutils.video import VideoStream
from imutils.video import WebcamVideoStream
from imutils.video import FPS
vs = WebcamVideoStream(src=0).start()
fps = FPS().start()

In [41]:
import imutils
import cv2
while True:
   # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=500)
 
    # grab the frame dimensions and convert it to a blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
        (300, 300), (104.0, 177.0, 123.0))
    
#     frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
#     cv2.namedWindow("preview")
    cv2.imshow("preview", frame)
    key = cv2.waitKey(3) & 0xFF
    fps.update()
 
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
                WebcamVideoStream(src=0).stop()
                vs.stream.release()
                break
    
    

In [29]:
vs.stop()
WebcamVideoStream(src=0).stop()
fps.stop()

In [30]:
vs.stream.release()

In [0]:
import matplotlib.pyplot as plt

In [66]:
from keras.models import model_from_json
network_json=model.to_json()
with open("style.json", "w") as json_file:
    json_file.write(network_json)
    
model.save_weights("styletransfer.h5")
print("Saved model to disk")

Saved model to disk
